In [1]:
import pandas as pd
pd.__version__

'2.2.3'

In [2]:
import numpy as np
print(np.__version__)

2.2.4


In [3]:
import sklearn
sklearn.__version__

'1.6.1'

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import root_mean_squared_error

### Q1. Downloading the data

In [5]:
df_train = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [6]:
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [7]:
# if error run:
#!pip install pyarrow # or fastparquet

In [8]:
len(df_train.columns)

19

### Q2. Computing duration

In [9]:
df_train.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [10]:
# compute the duration of a ride in minutes
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
df_train.duration = df_train.duration.apply(lambda td: td.total_seconds() / 60)
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [11]:
df_train.duration.std()

42.594351241920904

### Q3. Dropping outliers

In [12]:
number_records_initial = df_train.shape[0]
print(f"Number of records initially: {number_records_initial}")

Number of records initially: 3066766


In [13]:
df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]
number_records_left = df_train.shape[0]
print(f"Number of records left: {number_records_left}")
fraction_records_left = number_records_left / number_records_initial
print(f"Fraction of records left: {fraction_records_left:.2f}")

Number of records left: 3009173
Fraction of records left: 0.98


### Q4. One-hot encoding

In [14]:
categorical = ['PULocationID', 'DOLocationID']
df_train[categorical] = df_train[categorical].astype(str)
#df_train.info() # still shown as 'object'

In [15]:
train_dics = df_train[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dics)
X_train.shape[1]

515

### Q5. Training a model

In [16]:
target = 'duration'
y_train = df_train[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse_train = root_mean_squared_error(y_train, y_pred)
rmse_train

7.6492624397080675

### Q6. Evaluating the model

In [17]:
# preparing the validation data
# computing duration
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

In [18]:
# dropping outliers
number_records_initial = df_val.shape[0]
print(number_records_initial)
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
number_records_left = df_val.shape[0]
print(number_records_left)
fraction_records_left = number_records_left / number_records_initial
fraction_records_left

2913955
2855951


0.9800944077722545

In [19]:
# one-hot encoding
df_val[categorical] = df_val[categorical].astype(str)
val_dics = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dics)

In [20]:
# applying the model to validation data
y_val = df_val[target].values
y_pred = lr.predict(X_val)
rmse_val = root_mean_squared_error(y_val, y_pred)
rmse_val

7.81181211389241